# Deploy R Shiny Dashboard App

### Introduction

Now that we have prepared the data and stored it, we will now:

* Create a deployment space. 
* Store Shiny dashboard assets into the deployment space.
* Deploy Shiny assets as an app and view the dashboard.

Watson Machine Learning provides deployment spaces where the user can save, configure and deploy the shiny assets. The shiny app can be deployed in a similar way to models and functions, using the watson_machine_learning_client package.

All of the files associated with the dashboard are contained in a zip file which is stored in data assets. CSV files stored in the zip file were generated from the `synthetic` data source in ***Data-Preparation*** notebook. If the user would like to make changes to the dashboard, they can download the zip from data assets and upload it in the RStudio IDE. 

### User Inputs

The user can specify the name for the space and shiny app deployment.

In [1]:
# Specify a name for the space being created. 
space_name = 'Healthcare Services Location Optimization Space'

# Specify a name for the r shiny app being deployed. 
r_shiny_deployment_name='Healthcare_Services_Location_Optimization_Shiny_App'


In [2]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient
import os

token = os.environ['USER_ACCESS_TOKEN']

wml_credentials = {
   "token": token,
   "instance_id" : "openshift",
   "url": os.environ['RUNTIME_ENV_APSX_URL'],
   "version": "3.0.0"
}

client = WatsonMachineLearningAPIClient(wml_credentials)

### Create the Deployment Space

Create a new deployment space. Use the name of the space as specified in the user inputs cell above. The space name will be used in future to identify this space. If a space with specified space_name already exists, a new space with the same name is created. Set this deployment space as the default space.

In [3]:
# create the space and set it as default
space_meta_data = {
        client.spaces.ConfigurationMetaNames.NAME : space_name,
        client.spaces.ConfigurationMetaNames.TAGS : [{'value':'Healthcare Services Location Optimization Space tag'}]
}

stored_space_details = client.spaces.store(space_meta_data)

space_uid = stored_space_details['metadata']['guid']

client.set.default_space(space_uid)

'SUCCESS'

### Store the App

Create the associated metadata and store the dashboard zip file in the deployment space. 

In [4]:
# Meta_props to store assets in space 
meta_props = {
    client.shiny.ConfigurationMetaNames.NAME: "Healthcare_Services_Location_Optimization_Shiny_assets",
    client.shiny.ConfigurationMetaNames.DESCRIPTION: 'Store shiny assets in deployment space' # optional
}
app_details = client.shiny.store(meta_props, '/project_data/data_asset/healthcare_services_location_optimization-dashboard.zip')

Creating Shiny asset...
SUCCESS


### Deploy the App

Create the metadata for the Shiny deployment by providing  name, description, R-Shiny options and Hardware specifications. R-Shiny configuration provides options on whom you want to share the dashboard with, they are 1) anyone with the link 2) Authenticated users 3) Collaborators in this deployment space

In [5]:
# Deployment metadata.
deployment_meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: r_shiny_deployment_name,
    client.deployments.ConfigurationMetaNames.DESCRIPTION: 'Deploy Healthcare_Services_Location_Optimization dashboard',
    client.deployments.ConfigurationMetaNames.R_SHINY: { 'authentication': 'anyone_with_url' },
    client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: { 'name': 'S', 'num_nodes': 1}
}

# Create the deployment.
app_uid = client.shiny.get_uid(app_details)
rshiny_deployment = client.deployments.create(app_uid, deployment_meta_props)



#######################################################################################

Synchronous deployment creation for uid: '0418f05a-bf43-4de3-b38f-e445a4c68d1b' started

#######################################################################################


initializing......
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='fdc35131-a882-451b-b196-25db57580826'
------------------------------------------------------------------------------------------------




### Launch Shiny App
Now that the dashboard is deployed, it can be accessed through the web browser. The app URL can be found by navigating to the deployed app in the deployment space. 

Open the Navigation Menu, under ***Analytics*** select ***Analytics deployments -> Healthcare Services Location Optimization Space -> Deployments -> Healthcare_Services_Location_Optimization_Shiny_App*** to find the dashboard URL.

Alternatively, the path for the app URL can be found from the deployment metadata created in the previous cell. This path should be appended to the user's Cloud Pak for Data hostname to get the complete app URL. To get the path, run the cell below:

In [6]:
print("{HOSTNAME}"+rshiny_deployment['metadata']['href'] + '/r_shiny')

{HOSTNAME}/v4/deployments/fdc35131-a882-451b-b196-25db57580826/r_shiny
